# Задание 5 - 20 баллов

- Загрузить набор данных Lenta.ru с помощью пакета Corus
- Обучить LDA модель, постараться подобрать адекватные параметры (num_topics, passes, alpha, iterations…) - **4 балла**
- Визуализировать результаты работы LDA с помощью pyLDAvis - **2 балла**
- Посчитать внутренние метрики обученных моделей LDA (с разными параметрами) и сравнить, соответствует ли метрика визуальному качеству работы моделей - **2 балла**
- Обучить модель BigARTM, использовать не менее двух регуляризаторов, оценить качество с помощью метрик - **5 баллов**
- Реализовать визуализацию топиков BigARTM через pyLDAvis - **4 балла**

- Обеспечена воспроизводимость решения: зафиксированы random_state, ноутбук воспроизводится от начала до конца без ошибок - **2 балла**

- Соблюден code style на уровне pep8 и [On writing clean Jupyter notebooks](https://ploomber.io/blog/clean-nbs/)  - **1 балл**

Примечание: подбирать параметры теметической модели можно также, как и для любой другой модели - на кроссвалидации, ориентируясь на метрики качества

In [38]:
from corus import load_lenta
import os
import gensim

from typing import List

from itertools import product

import demoji
import re
import spacy

from tqdm import tqdm

from gensim import corpora

import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import matplotlib.colors as mcolors
from gensim.models.coherencemodel import CoherenceModel

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

import warnings, logging

from collections import Counter

NLP = spacy.load("ru_core_news_sm")
CHECK_POS = {'PUNCT', 'ADP', 'AUX', 'CCONJ', 'SCONJ'}
PATTERN_EMAIL = r'([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+'

SEED = 566
# data dir
DATA_DIR = "../data"
# path to the data
DATA_PATH = os.path.join(DATA_DIR, "lenta-ru-news.csv.gz")
# amount of texts to work with
N_TEXTS = 500

# Загрузка текста

Загрузим данные в директорию `../data/`

In [6]:
# ! wget https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz
# ! mv lenta-ru-news.csv.gz ../data/

## Читаем датасет

In [7]:
records = load_lenta(DATA_PATH)
next(records)

LentaRecord(
    url='https://lenta.ru/news/2018/12/14/cancer/',
    title='Названы регионы России с\xa0самой высокой смертностью от\xa0рака',
    text='Вице-премьер по социальным вопросам Татьяна Голикова рассказала, в каких регионах России зафиксирована наиболее высокая смертность от рака, сообщает РИА Новости. По словам Голиковой, чаще всего онкологические заболевания становились причиной смерти в Псковской, Тверской, Тульской и Орловской областях, а также в Севастополе. Вице-премьер напомнила, что главные факторы смертности в России — рак и болезни системы кровообращения. В начале года стало известно, что смертность от онкологических заболеваний среди россиян снизилась впервые за три года. По данным Росстата, в 2017 году от рака умерли 289 тысяч человек. Это на 3,5 процента меньше, чем годом ранее.',
    topic='Россия',
    tags='Общество',
    date=None
)

In [8]:
dataset_text_topics = [next(records) for i in range(N_TEXTS)]
dataset = [tt.text for tt in dataset_text_topics]
target = [tt.topic for tt in dataset_text_topics]
dataset[0], target[0]

('Австрийские правоохранительные органы не представили доказательств нарушения российскими биатлонистами антидопинговых правил. Об этом сообщил посол России в Вене Дмитрий Любинский по итогам встречи уполномоченного адвоката дипмиссии с представителями прокуратуры страны, передает ТАСС. «Действует презумпция невиновности. Каких-либо ограничений свободы передвижения для команды нет», — добавили в посольстве. Международный союз биатлонистов (IBU) также не будет применять санкции к российским биатлонистам. Все они продолжат выступление на Кубке мира. Полиция нагрянула в отель сборной России в Хохфильцене вечером 12 декабря. Как написал биатлонист Александр Логинов, их считают виновными в махинациях с переливанием крови. Биатлонисту Антону Шипулину, также попавшему в список, полиция нанесла отдельный визит: сейчас он тренируется отдельно в австрийском Обертиллахе. Обвинения спортсмен назвал бредом, а также указал на «охоту на ведьм» в мировом биатлоне. В Австрии прием допинга — уголовное п

## Предобработка

In [9]:
def normalize_text(text: str) -> str:
    checked_text = demoji.replace(text)
    checked_text = re.sub(PATTERN_EMAIL, '<EMAIL>', checked_text)
    return checked_text


def normalize_data(text_data: List[str]) -> List[str]:
    return list(map(normalize_text, text_data))


def tokenize_clean_stem(text: str) -> List[str]:
    text_NLPed = NLP(text)
    return [token.lemma_ for token in text_NLPed
            if token.pos_ not in CHECK_POS and not token.is_stop]


def tokenize_clean_stem_data(text_data: List[str]) -> List[List[str]]:
    return list(map(tokenize_clean_stem, text_data))


def pipeline_preprocessing(text_data: List[str]) -> List[List[str]]:
    data = normalize_data(text_data)
    return tokenize_clean_stem_data(data)


In [10]:
%%time
preprocessed_dataset = pipeline_preprocessing(dataset)

CPU times: user 27.6 s, sys: 26.8 ms, total: 27.6 s
Wall time: 27.7 s


In [11]:
preprocessed_dataset[0][:10], len(preprocessed_dataset[0])

(['австрийский',
  'правоохранительный',
  'орган',
  'представить',
  'доказательство',
  'нарушение',
  'российский',
  'биатлонист',
  'антидопинговый',
  'правило'],
 100)

## Получение N-grams

`Automatically detect common phrases – aka multi-word expressions, word n-gram collocations – from a stream of sentences.`

In [12]:
def get_ngrams(texts_out, bigram_mod, trigram_mod):
    texts_out = [bigram_mod[doc] for doc in texts_out]
    texts_out = [trigram_mod[bigram_mod[doc]] for doc in texts_out]
    return texts_out

In [13]:
bigram = gensim.models.Phrases(preprocessed_dataset, min_count=5, threshold=100)  # higher threshold fewer phrases.
bigram[preprocessed_dataset][0][:5], bigram[preprocessed_dataset][5][5:10]

(['австрийский',
  'правоохранительный_орган',
  'представить',
  'доказательство',
  'нарушение'],
 ['ценность', 'распространять', 'интернет', 'социальный_сеть', 'инициатива'])

Некоторы фразочки объединились, и в целом звучат логично: 'правоохранительный_орган', 'социальный_сеть'

In [14]:
trigram = gensim.models.Phrases(bigram[preprocessed_dataset], threshold=100)
# make Phraser == FrozenPhrases
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
tokenized_dataset = get_ngrams(preprocessed_dataset, bigram_mod, trigram_mod)
tokenized_dataset[0][:10]

['австрийский',
 'правоохранительный_орган',
 'представить',
 'доказательство',
 'нарушение',
 'российский',
 'биатлонист',
 'антидопинговый_правило',
 'сообщить',
 'посол']

In [15]:
flag = 10
for item in tokenized_dataset:
    for token in item:
        if token.count('_') > 2:
            print(token)
            flag -= 1
    if flag < 0:
        break

московский_патриархат_упц_мп
заговор_цель_ведение_деятельность
московский_патриархат_упц_мп
московский_патриархат_упц_мп
московский_патриархат_упц_мп
московский_патриархат_упц_мп
московский_патриархат_упц_мп
московский_патриархат_упц_мп
московский_патриархат_упц_мп
заговор_цель_ведение_деятельность
релиз_поступить_редакция_ленты.ру


## Создание словаря

В конце у нас: bag-of-words format = list of (token_id, token_count)

In [16]:
id2word = corpora.Dictionary(tokenized_dataset)
dict(id2word)[1], dict(id2word)[5]

('ibu', 'антидопинговый_правило')

In [17]:
corpus = [id2word.doc2bow(text) for text in tokenized_dataset]
corpus[0][:10]

[(0, 1),
 (1, 1),
 (2, 2),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 3),
 (9, 1)]

# LDA модель

In [37]:
%%time
nums_topics = [3, 5, 7]
alphas = ['symmetric', 'asymmetric']  # A-priori belief on document-topic distribution
passes = [25, 100]  # Number of passes through the corpus during training
iterations = [25,
              100]  # Maximum number of iterations through the corpus when inferring the topic distribution of a corpus

params_sets = list(product(nums_topics, alphas, passes, iterations))
param_names = ['num_topics', 'alpha', 'passes', 'iterations']

coherence_scores = dict()
perplexity_scores = dict()

for num_topics, alpha, passes, iterations in tqdm(params_sets):

    lda_model = gensim.models.ldamodel.LdaModel(
        corpus=corpus,
        id2word=id2word,
        num_topics=num_topics,
        random_state=SEED,
        update_every=1,  # Number of documents to be iterated through for each update
        chunksize=10,  # == batch size
        passes=passes,  # Number of passes through the corpus during training
        alpha=alpha,  # A-priori belief on document-topic distribution
        iterations=iterations,
        # Maximum number of iterations through the corpus when inferring the topic distribution of a corpus,
    )

    coherence_model_lda = CoherenceModel(model=lda_model,
                                         texts=tokenized_dataset,
                                         dictionary=id2word,
                                         coherence='c_v')
    coherence_score = coherence_model_lda.get_coherence()

    perplexity_score = lda_model.log_perplexity(corpus)

    coherence_scores[(num_topics, alpha, passes, iterations)] = coherence_score
    perplexity_scores[(num_topics, alpha, passes, iterations)] = perplexity_score

24it [05:39, 14.15s/it]

CPU times: user 5min 32s, sys: 6.45 s, total: 5min 38s
Wall time: 5min 39s


In [42]:
2**perplexity_score

0.0016068993276186954

Let's look at scores

In [46]:
df_coherence = pd.DataFrame(list(coherence_scores.items()), columns=['Params', 'Coherence'])
df_perplexity = pd.DataFrame(list(perplexity_scores.items()), columns=['Params', 'Log_Perplexity'])
df = pd.merge(df_coherence, df_perplexity, on='Params', how='outer')
df['Perplexity'] = df['Log_Perplexity'].apply(lambda x: 10**x)
df_sorted = df.sort_values(by=['Coherence'], ascending=[False], na_position='last')
display(df_sorted)

,Params,Coherence,Log_Perplexity,Perplexity
16,"(7, symmetric, 25, 25)",0.371087,-9.474258,3.355381e-10
18,"(7, symmetric, 100, 25)",0.368366,-9.154316,7.009452e-10
12,"(5, asymmetric, 25, 25)",0.363092,-9.264435,5.439577e-10
2,"(3, symmetric, 100, 25)",0.357287,-8.864147,1.367267e-09
10,"(5, symmetric, 100, 25)",0.353174,-9.031985,9.289974e-10
20,"(7, asymmetric, 25, 25)",0.348206,-9.489662,3.238454e-10
0,"(3, symmetric, 25, 25)",0.347581,-9.029243,9.348818e-10
6,"(3, asymmetric, 100, 25)",0.342348,-8.869061,1.351881e-09
4,"(3, asymmetric, 25, 25)",0.338784,-9.025110,9.438222e-10
8,"(5, symmetric, 25, 25)",0.338635,-9.257704,5.524537e-10


Перплексия везде маленькая, и примерно одинаковая (как и когерентность). Возьмем по когерентности лучшую модель.